In [1]:
# get the data set
from word_mapping import word_mapping
from tag_mapping import tag_mapping
from utils import *
from check import check
import pickle
from tqdm import tqdm
import torch
import torch.nn as nn
from HMM import *
import BiLSTM_CRF
from my_transformer import Transformer

# get the data set
from word_mapping import word_mapping
from tag_mapping import tag_mapping
from utils import *
from check import check
import pickle
from tqdm import tqdm
import torch
import torch.nn as nn

cn_train_data, cn_valid_data = get_data_set('Chinese')
cn_TagMapping = tag_mapping('Chinese')
cn_WordMapping = word_mapping(cn_train_data)
cn_train_data.get_tag_mapping(cn_TagMapping.encode_mapping)
cn_train_data.get_word_mapping(cn_WordMapping.encode_mapping)
cn_valid_data.get_tag_mapping(cn_TagMapping.encode_mapping)
cn_valid_data.get_word_mapping(cn_WordMapping.encode_mapping)

en_train_data, en_valid_data = get_data_set('English')
en_TagMapping = tag_mapping('English')
en_WordMapping = word_mapping(en_train_data)
en_train_data.get_tag_mapping(en_TagMapping.encode_mapping)
en_train_data.get_word_mapping(en_WordMapping.encode_mapping)
en_valid_data.get_tag_mapping(en_TagMapping.encode_mapping)
en_valid_data.get_word_mapping(en_WordMapping.encode_mapping)

cn_tag_size = cn_TagMapping.num_tag
cn_vocab_size = cn_WordMapping.num_word

en_tag_size = en_TagMapping.num_tag
en_vocab_size = en_WordMapping.num_word

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
with open("transformer/Chinese.pkl", "rb") as f:
    cn_model = pickle.load(f)

print('---------------- Chinese Result ------------------')
language = 'Chinese'
my_path = f"my_{language}_transformer_result.txt"
file = open(my_path, "w")
for words, tags in tqdm(cn_valid_data):
    mask = torch.ones(len(words)).unsqueeze(0).to(device)
    torch_words = torch.tensor(words, dtype=torch.long, device=device).unsqueeze(0)
    tags_pred = cn_model.predict(torch_words, mask)
    tags_pred = torch.flatten(tags_pred).tolist()
    words_decoded = cn_WordMapping.decode(words)
    tags_decoded = cn_TagMapping.decode(tags_pred)
    for i in range(len(words)):
        file.write(f"{words_decoded[i]} {tags_decoded[i]}\n")
    file.write("\n")
file.close()

gold_path = f"./NER/{language}/validation.txt"
check(language, gold_path, my_path)


with open("transformer/English.pkl", "rb") as f:
    en_model = pickle.load(f)


print('---------------- English Result ------------------')
language = 'English'
my_path = f"my_{language}_transformer_result.txt"
file = open(my_path, "w")
for words, tags in tqdm(en_valid_data):
    mask = torch.ones(len(words)).unsqueeze(0).to(device)
    torch_words = torch.tensor(words, dtype=torch.long, device=device).unsqueeze(0)
    tags_pred = en_model.predict(torch_words, mask)
    tags_pred = torch.flatten(tags_pred).tolist()
    words_decoded = en_WordMapping.decode(words)
    tags_decoded = en_TagMapping.decode(tags_pred)
    for i in range(len(words)):
        file.write(f"{words_decoded[i]} {tags_decoded[i]}\n")
    file.write("\n")
file.close()

gold_path = f"./NER/{language}/validation.txt"
check(language, gold_path, my_path)

---------------- Chinese Result ------------------


100%|██████████| 476/476 [00:05<00:00, 85.60it/s] 


              precision    recall  f1-score   support

      B-NAME     0.9820    0.9732    0.9776       112
      M-NAME     0.9059    0.9390    0.9222        82
      E-NAME     0.9818    0.9643    0.9730       112
      S-NAME     0.0000    0.0000    0.0000         0
      B-CONT     0.8966    0.9286    0.9123        28
      M-CONT     0.9298    1.0000    0.9636        53
      E-CONT     1.0000    1.0000    1.0000        28
      S-CONT     0.0000    0.0000    0.0000         0
       B-EDU     0.9700    0.8661    0.9151       112
       M-EDU     0.9140    0.9497    0.9315       179
       E-EDU     0.9714    0.9107    0.9401       112
       S-EDU     0.0000    0.0000    0.0000         0
     B-TITLE     0.8983    0.8714    0.8846       770
     M-TITLE     0.9002    0.9105    0.9053      1921
     E-TITLE     0.9226    0.9286    0.9256       770
     S-TITLE     0.0000    0.0000    0.0000         0
       B-ORG     0.9406    0.9185    0.9294       552
       M-ORG     0.9584    

100%|██████████| 3453/3453 [00:25<00:00, 137.79it/s]


              precision    recall  f1-score   support

       B-PER     0.9664    0.9777    0.9720      1617
       I-PER     0.9561    0.9611    0.9586      1156
       B-ORG     0.9720    0.9609    0.9664      1661
       I-ORG     0.9602    0.8958    0.9269       835
       B-LOC     0.9777    0.9724    0.9751      1668
       I-LOC     0.9186    0.9222    0.9204       257
      B-MISC     0.9654    0.9530    0.9591       702
      I-MISC     0.8964    0.9213    0.9087       216

   micro avg     0.9642    0.9570    0.9606      8112
   macro avg     0.9516    0.9455    0.9484      8112
weighted avg     0.9643    0.9570    0.9605      8112

